# Actividad 2:
# Agrupando más allá de las formas: clustering con DBSCAN y DBHSCAN

## Objetivo
Desarrollar la capacidad de aplicar, comparar y evaluar modelos de clustering basados en densidad (DBSCAN y DBHSCAN) sobre un conjunto de datos con clústeres complejos y ruido, validando los resultados con métricas objetivas y visualización.

**Datasets utilizados:**  
`Wine`

---

### Estructura del Notebook:
1. Metodología.
2. Configuración del entorno.
3. Definicion de funciones.
4. Uso de funciones y resultados.
5. Análisis de los resultados y reflexiones finales.

---

## 1. Metodología

### Flujo de trabajo

1. **Carga y preprocesamiento de datos:**
    - Se carga el dataset **Wine** y se escala.

2. **Busqueda de mejor eps y clustering con DBSCAN y HDBSCAN:**
    - Se grafica la curva de distancias para encontrar posibles valores de eps.
    - Se prueban 5 valores de eps alrededor del codo de la curva evaluando su silueta y db index para elegir y usar el mejor.
    - Se aplica HDBSCAN.


3. **Visualización e interpretación:**
    - Gráfico de curva de distancias.
    - Resumen de la silueta y db index de los 5 valores probados en DBSCAN y del resultado de HDBSCAN.
    - Gráficos representando en 2D los resultados de ambos métodos de clustering.

---

# 2. Configuración del entorno

--- 

In [ ]:
import warnings

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.datasets import load_wine, make_moons
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score

import hdbscan

warnings.filterwarnings("ignore", category=FutureWarning)

# 3. Definición de funciones

> **Nota:** Para mejor comprensión de las funciones y su utilidad, esta sección se divide en bloques, en donde cada uno responde a una parte diferente de la metodología de trabajo. 

---

**Bloque 1:** Carga y preprocesamiento de datos.

- **`carga_y_preprocesamiento()`** 
Carga, escala y aplica PCA al dataset.

---

`Justificación del uso del dataset Wine`

Si bien el trabajo daba la opción a elegir entre 3 datasets, Wine fue elegido debido a las siguientes razones:

- Datos reales y multidimensionales: A diferencia de los datasets sintéticos como make_moons o make_blobs, Wine contiene 13 variables reales derivadas de análisis químicos de vinos. Esto permite un análisis más realista y relevante en escenarios de clustering.

- Aplicación de técnicas de reducción de dimensionalidad: Con Wine es posible aplicar PCA para explorar la estructura interna de datos de alta dimensión, lo cual no sería significativo con make_moons (2D) o make_blobs (datos generados artificialmente sin correlación real).

- Evaluación más completa: Al tener datos complejos, es posible evaluar cómo algoritmos como DBSCAN y HDBSCAN se comportan con clusters de densidad y distribución variable, proporcionando un análisis más rico que con datasets sintéticos diseñados para formas simples.

---

`Justificación del número de componentes PCA`

Para seleccionar el número adecuado de componentes principales se evaluó la varianza explicada acumulada con 2, 3 y 4 componentes:

- Con 2 componentes, la varianza acumulada es aproximadamente 55%, lo cual es bajo y podría no capturar suficiente información relevante del dataset.

- Con 3 componentes, la varianza acumulada aumenta a cerca de 66%, ofreciendo un mejor balance entre reducción dimensional y conservación de información.

- Con 4 componentes, la varianza acumulada alcanza cerca del 73%, pero al usar este número se observó que el algoritmo HDBSCAN presenta una silueta muy baja (menor a 0.1), indicando una agrupación menos coherente.

Por lo tanto, se decidió usar 3 componentes principales, ya que permiten conservar una proporción significativa de la varianza y a la vez mantienen un rendimiento adecuado en los algoritmos de clustering evaluados.

---

In [ ]:
def carga_y_preprocesamiento():
    """
    Carga el dataset Wine desde scikit-learn y aplica escalado estándar a las variables numéricas.

    Returns:
        X (numpy.ndarray): Matriz de características original sin escalar.
        X_scaled (numpy.ndarray): Matriz de características escalada con media 0 y desviación estándar 1.
    """
    data = load_wine()
    X = data.data

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # PCA
    pca = PCA(n_components=3)
    X_pca = pca.fit_transform(X_scaled)
    
    return X_pca

**Bloque 2:** Busqueda de mejor eps y clustering con DBSCAN y HDBSCAN.

- **`plot_k_distance_curve()`** 
Grafica la curva de distancia con k = 4.

- **`dbscan_param_search()`** 
Evalúa valores de posibles eps extraídos del gráfico de curva de distancias para usar el mejor.

- **`hdbscan_cluster()`** 
Crea el modelo con HDBSCAN.

---

`Justificación del umbral n_clusters > 1 para Silhouette y Davies-Bouldin en evaluación de eps`

- Las métricas Silhouette y Davies-Bouldin Index están diseñadas para evaluar la calidad de la separación entre clústeres.

- Silhouette Score compara la distancia promedio de cada punto con los puntos de su propio clúster frente a los puntos del clúster más cercano.
Si existe solo un clúster, no hay un "clúster vecino" con el cual hacer esta comparación, lo que hace imposible calcular la métrica.

- Davies-Bouldin Index se basa en la separación entre cada par de clústeres.
Con un único clúster, no existen pares que permitan medir dispersión ni distancia entre grupos.

- Por estas razones, cuando n_clusters <= 1 (es decir, solo hay un clúster o todos los puntos fueron clasificados como ruido), estas métricas no se calculan y se devuelven como None, ya que carecen de sentido en dicho contexto.

---

`Justificación del uso de k = 4 y min_samples = 4`

- En el algoritmo DBSCAN, el parámetro min_samples define el número mínimo de puntos necesarios para que una región sea considerada un clúster denso. Para estimar el valor de eps, se utiliza la curva de distancia al k-ésimo vecino, donde el valor de k debe ser coherente con min_samples.

- Regla práctica: Los autores de DBSCAN recomiendan usar valores pequeños, típicamente entre 3 y 5, ya que estos permiten detectar agrupaciones pequeñas sin requerir densidades excesivamente altas.

- Relación con la dimensionalidad: Se suele tomar min_samples ≈ D + 1, donde D es el número de dimensiones tras la reducción con PCA. En nuestro caso, con 3 componentes principales, el valor min_samples = 4 (y por tanto k = 4) es una elección natural y balanceada.

- Consistencia con la curva k-distancia: Probar con valores mayores no aportó ventajas significativas, mientras que k = 4 ofreció una curva clara y un buen punto de codo para seleccionar eps.

---

In [ ]:
def plot_k_distance_curve(X, k=4):
    """
    Grafica la curva de distancia al k-ésimo vecino para estimar eps en DBSCAN.
    
    Parámetros:
        X: array-like, datos preprocesados (p. ej. tras PCA)
        k: entero, número de vecinos (min_samples típico)
    """
    nn = NearestNeighbors(n_neighbors=k)
    nn.fit(X)
    distances, _ = nn.kneighbors(X) # "_" representa los índices de los vecino, no se necesitan para la curva
    k_distances = np.sort(distances[:, k-1])
    
    plt.figure(figsize=(8,4))
    plt.plot(k_distances)
    plt.xlabel("Puntos ordenados")
    plt.ylabel(f"Distancia al {k}º vecino")
    plt.title("Curva de distancia para estimar eps")
    plt.grid(True)
    plt.show()

def dbscan_param_search(X, eps_values, min_samples=4):
    """
    Aplica DBSCAN con distintos eps, evalúa índices y devuelve resultados.
    
    Parámetros:
        X: array-like, datos preprocesados
        eps_values: lista o array de floats, valores de eps a probar
        min_samples: int, parámetro min_samples de DBSCAN
    
    Retorna:
        resultados: lista de dicts con eps, n_clusters, silhouette, db_index, labels
    """
    resultados = []
    
    for eps in eps_values:
        db = DBSCAN(eps=eps, min_samples=min_samples).fit(X)
        labels = db.labels_
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        
        if n_clusters > 1:
            sil_score = silhouette_score(X, labels)
            db_score = davies_bouldin_score(X, labels)
        else:
            sil_score = None
            db_score = None
        
        resultados.append({
            'eps': eps,
            'n_clusters': n_clusters,
            'silhouette': sil_score,
            'db_index': db_score,
            'labels': labels
        })
        
    return resultados

def hdbscan_cluster(X, min_cluster_size=5):
    """
    Aplica HDBSCAN y devuelve etiquetas y métricas.
    
    Parámetros:
        X: array-like, datos preprocesados
        min_cluster_size: int, tamaño mínimo del cluster para HDBSCAN
    
    Retorna:
        labels: array de etiquetas
        silhouette: float o None
        db_index: float o None
    """
    clusterer = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size)
    labels = clusterer.fit_predict(X)
    
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    
    if n_clusters > 1:
        sil_score = silhouette_score(X, labels)
        db_score = davies_bouldin_score(X, labels)
    else:
        sil_score = None
        db_score = None
    
    return labels, sil_score, db_score

**Bloque 3:** Generacion y visualización de resultados con PCA y TSNE.

- **`plot_clusters_pca()`** 
Genera una figura con 2 gráficos correspondientes a la visualización 2d usando PCA.

- **`plot_clusters_tnse()`** 
Genera una figura con 2 gráficos correspondientes a la visualización 2d usando TSNE.

---

`Justificación para usar solo 2 componentes en la visualización (PCA y t-SNE)`

En el análisis principal se utilizan 4 componentes principales (PCA) para capturar una mayor cantidad de varianza y representar mejor la estructura interna de los datos. Esto ayuda a conservar más información relevante para el clustering.

Sin embargo, la representación gráfica debe limitarse a dos dimensiones para facilitar la interpretación visual. Por eso, se utilizan dos técnicas diferentes para reducir la dimensionalidad a 2D para la visualización:

- PCA 2D:
Seleccionamos las primeras dos componentes principales directamente del resultado PCA de 4 componentes. Estas componentes explican la mayor parte de la varianza acumulada y permiten observar la estructura general y la separación de los clusters en un espacio lineal. Es una forma rápida, lineal y consistente de representar los datos para visualización.

- t-SNE 2D:
Como técnica no lineal y estocástica, t-SNE transforma los datos ya reducidos (en este caso, las 4 componentes PCA) a un espacio bidimensional buscando preservar relaciones de proximidad local. Esto puede mostrar agrupamientos y estructuras complejas que PCA lineal no detecta, aunque puede distorsionar la forma global y las distancias absolutas.
Esta transformación adicional ayuda a visualizar mejor clusters con formas irregulares o con separaciones no lineales.

Por lo tanto, combinar ambas visualizaciones ofrece una comprensión más completa: la visualización PCA para la estructura general, y la visualización t-SNE para detalles finos y separación local de clusters.

---

In [ ]:
def plot_clusters_pca(X_pca, labels_db, labels_hdb, eps):
    """
    Muestra en una sola figura dos gráficos comparando DBSCAN y HDBSCAN.
    
    Parámetros:
        X_2d: array-like, datos con 2 dimensiones (ej. PCA con 2 comps)
        labels_db: array-like, etiquetas de cluster de DBSCAN
        labels_hdb: array-like, etiquetas de cluster de HDBSCAN
        eps: float, valor de eps usado en DBSCAN
    """
    # Asegurarse de que, independiente del numero de componentes, X_pca muestre solo 2 dimensiones
    X_pca_2d = X_pca[:, :2]

    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # --- DBSCAN ---
    unique_labels_db = np.unique(labels_db)
    palette_db = sns.color_palette('tab10', len(unique_labels_db))
    colors_db = [palette_db[i] if i != -1 else (0, 0, 0) for i in labels_db]

    axes[0].scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=colors_db, s=50, alpha=0.7)
    axes[0].set_title(f"DBSCAN (eps={eps})")
    axes[0].set_xlabel("Componente 1")
    axes[0].set_ylabel("Componente 2")

    # --- HDBSCAN ---
    unique_labels_hdb = np.unique(labels_hdb)
    palette_hdb = sns.color_palette('tab10', len(unique_labels_hdb))
    colors_hdb = [palette_hdb[i] if i != -1 else (0, 0, 0) for i in labels_hdb]

    axes[1].scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=colors_hdb, s=50, alpha=0.7)
    axes[1].set_title("HDBSCAN")
    axes[1].set_xlabel("Componente 1")
    axes[1].set_ylabel("Componente 2")

    plt.tight_layout()
    plt.show()


def plot_clusters_tsne(X_pca, labels_db, labels_hdb, eps):
    """
    Muestra en una sola figura dos gráficos comparando DBSCAN y HDBSCAN.
    
    Parámetros:
        X_pca: array-like, datos reducidos con PCA (ej. 4 componentes)
        labels_db: array-like, etiquetas de cluster de DBSCAN
        labels_hdb: array-like, etiquetas de cluster de HDBSCAN
        eps: float, valor de eps usado en DBSCAN
    """
    # Reducir a 2D con t-SNE para mejor visualización
    tsne = TSNE(n_components=2, random_state=42)
    X_tsne_2d = tsne.fit_transform(X_pca)

    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # --- DBSCAN ---
    unique_labels_db = np.unique(labels_db)
    palette_db = sns.color_palette('tab10', len(unique_labels_db))
    colors_db = [palette_db[i] if i != -1 else (0, 0, 0) for i in labels_db]

    axes[0].scatter(X_tsne_2d[:, 0], X_tsne_2d[:, 1], c=colors_db, s=50, alpha=0.7)
    axes[0].set_title(f"DBSCAN (eps={eps})")
    axes[0].set_xlabel("Dimensión 1 (t-SNE)")
    axes[0].set_ylabel("Dimensión 2 (t-SNE)")

    # --- HDBSCAN ---
    unique_labels_hdb = np.unique(labels_hdb)
    palette_hdb = sns.color_palette('tab10', len(unique_labels_hdb))
    colors_hdb = [palette_hdb[i] if i != -1 else (0, 0, 0) for i in labels_hdb]

    axes[1].scatter(X_tsne_2d[:, 0], X_tsne_2d[:, 1], c=colors_hdb, s=50, alpha=0.7)
    axes[1].set_title("HDBSCAN")
    axes[1].set_xlabel("Dimensión 1 (t-SNE)")
    axes[1].set_ylabel("Dimensión 2 (t-SNE)")

    plt.tight_layout()
    plt.show()

**Bloque 4:** Función de ejecución del código.

- **`main()`** 
Ejecuta el flujo completo de análisis y visualización de clustering con DBSCAN y HDBSCAN.

In [ ]:
def main():
    """
    Función principal que orquesta la carga, preprocesamiento, análisis PCA, 
    optimización de hiperparámetros, entrenamiento, evaluación y visualización 
    de modelos KNN con y sin reducción dimensional mediante PCA sobre el dataset Wine.
    """
    # 1. Cargar y preprocesar (escalar, PCA 4 comps)
    X_pca = carga_y_preprocesamiento()

    # 2. Graficar curva k-distancia para estimar eps
    print("Curva de distancia al k-ésimo vecino para estimar eps")
    plot_k_distance_curve(X_pca, k=4)

    # 3. Buscar mejor eps para DBSCAN
    eps_range = [0.7, 0.75, 0.8, 0.85, 0.9]  # o el rango que decidas
    resultados_dbscan = dbscan_param_search(X_pca, eps_range, min_samples=4)

    # Mostrar resumen sin etiquetas
    print("\nResultados de evaluación de valores eps en DBSCAN:")
    for r in resultados_dbscan:
        resumen = {k: v for k, v in r.items() if k != 'labels'}  # no mostrar labels
        print(resumen)

    # 4. Elegir el mejor eps (por ejemplo 0.85) y obtener sus labels
    best_eps = 0.85
    best_labels = [r['labels'] for r in resultados_dbscan if r['eps'] == best_eps][0]

    # 5. Aplicar HDBSCAN
    labels_hdbscan, sil_hdbscan, db_hdbscan = hdbscan_cluster(X_pca)

    print("\nResultados de HDBSCAN:")
    print(f"Clusters: {len(set(labels_hdbscan)) - (1 if -1 in labels_hdbscan else 0)}, "
          f"Silhouette: {sil_hdbscan}, DB Index: {db_hdbscan}")

    # 6. Visualizar resultados con los 2 primeros componentes PCA
    print("\nVicualización de resultados con PCA y t-SNE")
    plot_clusters_pca(X_pca, best_labels, labels_hdbscan, best_eps)
    plot_clusters_tsne(X_pca, best_labels, labels_hdbscan, best_eps)

# 4. Visualización de resultados

Se muestran los resultados obtenidos a partir de la ejecución de la funcion **main()**.

---

In [ ]:
if __name__ == "__main__":
    main()

# 5. Análisis de los resultados y reflexiones finales

---

### Elección del parámetro eps en DBSCAN y análisis de silueta y db index

La elección del parámetro eps = 0.85 se fundamenta en un balance entre la calidad interna de los clusters y la cantidad razonable de grupos generados:

- Aunque valores mayores de eps, como 0.9, presentan un índice de silueta más alto (0.28), la cantidad de clusters cae a solo 2, lo que limita el detalle del agrupamiento.

- Por otro lado, valores menores como 0.7 generan más clusters (9) pero con una silueta mucho más baja (0.07), lo que indica agrupaciones menos definidas.

- Con eps = 0.85 se obtuvo un índice de silueta intermedio-alto (0.24) con 4 clusters, lo cual representa un compromiso adecuado para capturar estructura en los datos sin perder cohesión interna ni generar un número excesivo de grupos poco significativos.

Por otro lado, HDBSCAN produjo 3 clusters con un índice de silueta ligeramente menor (0.20), lo que refleja agrupamientos algo menos cohesionados. Sin embargo, HDBSCAN es un algoritmo que maneja mejor ruido y clusters de forma irregular, y su capacidad para identificar puntos considerados ruido (-1 en etiquetas) puede hacer que el número efectivo de clusters sea menor, pero de mayor calidad en términos de diferenciación natural en los datos.

En términos del índice Davies-Bouldin (DB index), un valor menor indica clusters más separados y compactos. Aquí, HDBSCAN mostró un DB index moderadamente mejor (1.81) que DBSCAN con eps=0.85 (2.11), sugiriendo que aunque la silueta sea menor, la separación entre clusters es mejor con HDBSCAN. Esto puede indicar que HDBSCAN está formando clusters con límites más claros, aunque internamente sean menos densos o compactos que los clusters de DBSCAN.

En resumen, DBSCAN con eps=0.85 ofrece un equilibrio entre cantidad de clusters y cohesión interna, mientras que HDBSCAN proporciona una agrupación que puede ser más robusta a ruido y estructuras irregulares, con mejor separación pero menor cohesión interna.

---

### Consideraciones sobre la visualización en 2D (PCA vs t-SNE)

Para visualizar los clusters se redujo la dimensionalidad a dos dimensiones utilizando dos métodos distintos:

- **PCA (2 componentes principales):** conserva la mayor varianza lineal posible y muestra los clusters distribuidos con formas más alineadas a los ejes, formando una estructura tipo "V", con grupos ubicados en diferentes cuadrantes.

- **t-SNE aplicado sobre PCA:** genera una representación no lineal que agrupa mejor los puntos similares, mostrando clusters más compactos y separados espacialmente en el plano 2D, con una distribución que refleja mejor las relaciones locales en los datos, aunque con escalas y rangos más amplios y menos interpretables en términos absolutos.

Esta diferencia explica por qué los clusters pueden verse distribuidos de manera distinta en cada visualización, siendo t-SNE útil para resaltar agrupamientos naturales no lineales, mientras que PCA es más útil para una interpretación general y lineal de la estructura.

---

## Reflexiones finales

La elección del método depende del objetivo del análisis: si se prioriza la robustez frente a ruido y clusters con formas irregulares, HDBSCAN puede ser la mejor opción; mientras que para obtener agrupamientos más definidos y un número controlado de clusters, DBSCAN con eps=0.85 resulta más adecuado.

### ¿Qué algoritmo funcionó mejor?

El algoritmo DBSCAN con eps = 0.85 logró el mejor balance en cuanto a cohesión (silhouette 0.24) y cantidad adecuada de clusters (4), superando en silueta a HDBSCAN (0.20) y manteniendo un número razonable de grupos. Aunque HDBSCAN muestra un DB index menor que algunos valores de DBSCAN, su silueta es inferior a la mejor configuración DBSCAN.

### Limitaciones encontradas:

La principal limitación fue la sensibilidad a la selección del parámetro eps en DBSCAN, que requiere una búsqueda cuidadosa para evitar obtener solo un cluster o demasiados clusters poco coherentes. Además, la elección del número de componentes PCA afecta significativamente la calidad de los resultados, y no siempre aumentar la dimensionalidad mejora el agrupamiento.

### Posibles mejoras:

Para futuras iteraciones se recomienda explorar técnicas de selección automática de eps más robustas, o combinar DBSCAN con métodos de reducción dimensional que preserven mejor la estructura, como t-SNE o UMAP. También se podría investigar un análisis más profundo de la estabilidad de clusters y validación externa si hubiera etiquetas disponibles.